#### Printing out the 0th moment (from raw) and column density (from usharp30) images for both the 1-8 and the 1-6 sets. 5-95 cut on the intensity

In [1]:
import pickle
import numpy as np
from cube_fil_finder.util import cube_moments
from cube_fil_finder.galfa import galfa_const
from cube_fil_finder.galfa import galfa_util
from cube_fil_finder.util import moments

from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
reload(cube_moments)
reload(galfa_util)

<module 'cube_fil_finder.galfa.galfa_util' from '/Users/larryli/Documents/CC/16-17/research/3d_filfind/cube_fil_finder/galfa/galfa_util.pyc'>

In [3]:
ex_galfa_slice_path = '/Volumes/LarryExternal1/Research_2017/GALFA_slices_backup/umask_gaussian_30/GALFA_HI_W_S0955_V-050.4kms_umask.fits'

In [5]:
pickle_path_all_trees_1_8 = '../../pickled_dicts/fourth_batch/all_trees_1_8.p'  
all_trees = pickle.load(open(pickle_path_all_trees_1_8, 'rb'))

In [6]:
pickle_path_all_data = '../../pickled_dicts/fourth_batch/all_trees_1_8_prop.p'
all_prop = pickle.load(open(pickle_path_all_data, 'rb'))

In [7]:
bad_trees = []
for k in all_prop:
    if len(all_prop[k]) == 0:
        bad_trees.append(k)

In [8]:
for k in bad_trees:
    all_trees.pop(k)

In [9]:
len(all_trees)

1095

In [ ]:
for k in all_trees:
    tree = all_trees[k]

    tree_mask = tree.root_node.mask
    tree_size = tree.root_node.masked_area_size

    ra, dec = galfa_util.galfa_index_to_radecs([tree.root_node.corners[0][0], tree.root_node.corners[1][0]],
                                               [tree.root_node.corners[0][1], tree.root_node.corners[1][1]])

    tree_data_cube_umask30 = galfa_util.get_galfa_data_cube_from_tree(tree, cube_type='umask30')
    moment_0_map_umask30 = cube_moments.moment_0_from_cube(tree_data_cube_umask30)
    column = cube_moments.column_density_from_moment_0_map(moment_0_map_umask30)

    tree_data_cube_raw = galfa_util.get_galfa_data_cube_from_tree(tree, cube_type='raw')
    moment_0_map_raw = cube_moments.moment_0_from_cube(tree_data_cube_raw)

    fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

    axs = [ax1, ax2]
    maps = [column, moment_0_map_raw]

    for i in range(len(axs)):
        cutoff = np.percentile(maps[i], 5)
        cutoff2 = np.percentile(maps[i], 95)
        im = axs[i].imshow(maps[i].clip(cutoff, cutoff2), origin='lower', cmap='binary') #imshow does m by n
        axs[i].contour(tree_mask, alpha=.5, colors='red', linewidths=.3)

        axs[i].set_xticks([0, tree.root_node.corners[1][0]-tree.root_node.corners[0][0]])
        axs[i].set_xticklabels(np.round(ra,2))
        axs[i].set_yticks([0, tree.root_node.corners[1][1]-tree.root_node.corners[0][1]])
        axs[i].set_yticklabels(np.round(dec,2))
        axs[i].set_xlabel('RA')
        axs[i].set_ylabel('DEC')

        if i == 0:
            axs[i].set_title('column from umask30')
        elif i == 1:
            axs[i].set_title('raw moment 0')
        fig.colorbar(im, ax=axs[i], shrink=.8)

    fig.suptitle('{0}'.format(k))
    fig.tight_layout()
    fig.savefig('../../vis/column_plots_fourth_batch/{0}.pdf'.format(k))